In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.



In [2]:
df = pd.read_csv('Final')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
#lets first create a map of Toronto with neighborhoods superimposed on top
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


In [5]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [6]:
#Foursquare Credentials
CLIENT_ID = 'PHYK1JWWRGJ1YDTYJ4OFXGZWYPY2K4Q2DTHNAS3XXGSDQNU4' # your Foursquare ID
CLIENT_SECRET = 'JMJWYT4DJSF4N0DWNRHGAJA1WVKZUJ5ISKBJBAYWWYKIZ0QU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PHYK1JWWRGJ1YDTYJ4OFXGZWYPY2K4Q2DTHNAS3XXGSDQNU4
CLIENT_SECRET:JMJWYT4DJSF4N0DWNRHGAJA1WVKZUJ5ISKBJBAYWWYKIZ0QU


In [12]:
#dataframe that contains neighborhoods with substring toronto
Toronto_data = df[df['Borough'].str.contains('Central Toronto')].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [11]:
#lets explore the neighborhood of Central Toronto
address = 'Central Toronto, ON, Canada'

geolocator = Nominatim(user_agent="CT_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


In [15]:
# create map of Central Toronto using latitude and longitude values
map_CT = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CT)  
    
map_CT

In [16]:
#create a function to explore all neighborhoods of central toronto
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
CentralToronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville


In [18]:
print(CentralToronto_venues.shape)
CentralToronto_venues.head()

(114, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


In [20]:
CentralToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,35,35,35,35,35,35
Davisville North,7,7,7,7,7,7
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",2,2,2,2,2,2
North Toronto West,20,20,20,20,20,20
Roselawn,3,3,3,3,3,3
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",24,24,24,24,24,24


In [21]:
print('There are {} uniques categories.'.format(len(CentralToronto_venues['Venue Category'].unique())))

There are 66 uniques categories.


In [22]:
# one hot encoding
CentralToronto_onehot = pd.get_dummies(CentralToronto_venues[['Venue Category']], prefix="", prefix_sep="")
#manhattan_onehot.head()
# add neighborhood column back to dataframe
CentralToronto_onehot['Neighborhood'] = CentralToronto_venues['Neighborhood'] 
#manhattan_onehot.head()

# move neighborhood column to the first column
fixed_columns = [CentralToronto_onehot.columns[-1]] + list(CentralToronto_onehot.columns[:-1])
#fixed_columns
CentralToronto_onehot = CentralToronto_onehot[fixed_columns]

CentralToronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
CentralToronto_onehot.shape

(114, 67)

In [25]:
CentralToronto_grouped = CentralToronto_onehot.groupby('Neighborhood').mean().reset_index()
CentralToronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.0000,0.0000,0.028571,0.000000,0.028571,0.000000,0.000000,0.085714,0.00,0.00,0.057143,0.000000,0.028571,0.000000,0.085714,0.028571,0.000000,0.028571,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.028571,0.028571,0.028571,0.057143,0.00,0.000000,0.000000,0.000000,0.028571,0.057143,0.00,0.0000,0.000000,0.00,0.000000,0.000000,0.028571,0.028571,0.085714,0.0,0.000000,0.00,0.028571,0.00,0.085714,0.028571,0.00,0.00,0.00,0.0000,0.0000,0.057143,0.000000,0.0,0.028571,0.028571,0.00,0.000000,0.0000,0.00
1,Davisville North,0.000000,0.000000,0.0000,0.0000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.0000,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.142857,0.000000,0.00,0.00,0.00,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.00
2,Forest Hill North & West,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.0000,0.250000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.0000,0.00
3,Lawrence Park,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.0000,0.000000,0.333333,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.00
4,"Moore Park, Summerhill East",0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.0000,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.000000,0.0000,0.00
5,North Toronto West,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.05,0.15,0.100000,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.05,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.05,0.000000,0.000000,0.050000,0.0

In [26]:
CentralToronto_grouped.shape

(9, 67)

In [27]:
num_top_venues = 5

for hood in CentralToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CentralToronto_grouped[CentralToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
              venue  freq
0    Sandwich Place  0.09
1       Pizza Place  0.09
2              Café  0.09
3      Dessert Shop  0.09
4  Sushi Restaurant  0.06


----Davisville North----
               venue  freq
0              Hotel  0.14
1     Breakfast Spot  0.14
2   Department Store  0.14
3  Food & Drink Shop  0.14
4                Gym  0.14


----Forest Hill North & West----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2   Mexican Restaurant  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1          Swim School  0.33
2                 Park  0.33
3  Rental Car Location  0.00
4                  Pub  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Playground   0.5
1         Tennis Court   0.5
2  American Restaurant   0.0
3          Music Venue   0.0
4  Rental Car Location   0.0


----North Toron

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CentralToronto_grouped['Neighborhood']

for ind in np.arange(CentralToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CentralToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Café,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Gas Station,Farmers Market
1,Davisville North,Department Store,Breakfast Spot,Hotel,Food & Drink Shop,Park,Sandwich Place,Gym,Brewery,Farmers Market,BBQ Joint
2,Forest Hill North & West,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop
3,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
4,"Moore Park, Summerhill East",Playground,Tennis Court,Yoga Studio,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Furniture / Home Store


In [30]:
# set number of clusters
kclusters = 5

CentralToronto_grouped_clustering = CentralToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CentralToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 4, 2, 0, 1, 0, 0], dtype=int32)

In [33]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CentralToronto_merged = Toronto_data
CentralToronto_merged = CentralToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

CentralToronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Park,Swim School,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Department Store,Breakfast Spot,Hotel,Food & Drink Shop,Park,Sandwich Place,Gym,Brewery,Farmers Market,BBQ Joint
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Shoe Store,Health & Beauty Service,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Gas Station,Farmers Market
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Playground,Tennis Court,Yoga Studio,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Furniture / Home Store


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CentralToronto_merged['Latitude'], CentralToronto_merged['Longitude'], CentralToronto_merged['Neighborhood'], CentralToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
#cluster 1
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 0, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,0,Department Store,Breakfast Spot,Hotel,Food & Drink Shop,Park,Sandwich Place,Gym,Brewery,Farmers Market,BBQ Joint
2,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Shoe Store,Health & Beauty Service,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park
3,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Gas Station,Farmers Market
5,Central Toronto,0,Coffee Shop,Pub,American Restaurant,Pizza Place,Bagel Shop,Bank,Vietnamese Restaurant,Fried Chicken Joint,Sushi Restaurant,Supermarket
8,Central Toronto,0,Sandwich Place,Café,Coffee Shop,American Restaurant,Furniture / Home Store,History Museum,Indian Restaurant,Flower Shop,Liquor Store,Donut Shop


In [38]:
#cluster 2
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 1, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,1,Home Service,Garden,Music Venue,Dessert Shop,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint


In [39]:
#cluster 3
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 2, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Playground,Tennis Court,Yoga Studio,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Furniture / Home Store


In [41]:
#cluster 4
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 3, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,3,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop


In [42]:
#cluster 5
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 4, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,4,Bus Line,Park,Swim School,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint


In [ ]:
#End of the Lab